# Importing Modules

In [826]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [827]:
# Setting Environment to ignore future warnings
import warnings
warnings.simplefilter(action="ignore")

In [828]:
# Setting pandas default parameters
pd.set_option("max_rows", 85)
pd.set_option("max_columns", 85)

# Loading Data

In [829]:
dataset_path="datasets/houseprices/"

In [830]:
df_train = pd.read_csv(dataset_path + "train.csv", index_col='Id')
df_test = pd.read_csv(dataset_path + "test.csv", index_col='Id')
main_data = pd.concat([df_train, df_test])
main_data.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706.0,Unf,0.0,150.0,856.0,GasA,Ex,Y,SBrkr,856,854,0,1710,1.0,0.0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2.0,548.0,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500.0
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978.0,Unf,0.0,284.0,1262.0,GasA,Ex,Y,SBrkr,1262,0,0,1262,0.0,1.0,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2.0,460.0,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500.0
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486.0,Unf,0.0,434.0,920.0,GasA,Ex,Y,SBrkr,920,866,0,1786,1.0,0.0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2.0,608.0,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500.0
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216.0,Unf,0.0,540.0,756.0,GasA,Gd,Y,SBrkr,961,756,0,1717,1.0,0.0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3.0,642.0,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000.0
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655.0,Unf,0.0,490.0,1145.0,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1.0,0.0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3.0,836.0,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000.0


In [831]:
main_data.shape

(2919, 80)

# Outlier Detection and Removal

Because we only have to check and remove outliers. So, we will deal only with Numerical features. Let's extract only numerical features from data.

In [832]:
# num_cols = ["MSSubClass", "LotFrontage", "YearBuilt", "LotArea", "MasVnrArea", "BsmtFinSF1", "BsmtFinSF2", "BsmtUnfSF", "TotalBsmtSF","GarageYrBlt", "1stFlrSF", "2ndFlrSF", "GrLivArea", "GarageArea", "WoodDeckSF", "OpenPorchSF", "EnclosedPorch", "3SsnPorch", "ScreenPorch", "PoolArea", "MiscVal", "MoSold", "YrSold"]
numeric_columns = main_data.select_dtypes(include=np.number).columns.tolist()
num_cols = [nc for nc in numeric_columns if main_data[nc].nunique()>10]

df = main_data[num_cols]
del df['SalePrice']
num_cols = df.columns
# df = df.loc[:, (df==0).mean() < .5]
# num_cols = df.columns
df.head()

,MSSubClass,LotFrontage,LotArea,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,TotRmsAbvGrd,GarageYrBlt,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold
Id,,,,,,,,,,,,,,,,,,,,,,,,,
1,60,65.0,8450,2003,2003,196.0,706.0,0.0,150.0,856.0,856,854,0,1710,8,2003.0,548.0,0,61,0,0,0,0,0,2
2,20,80.0,9600,1976,1976,0.0,978.0,0.0,284.0,1262.0,1262,0,0,1262,6,1976.0,460.0,298,0,0,0,0,0,0,5
3,60,68.0,11250,2001,2002,162.0,486.0,0.0,434.0,920.0,920,866,0,1786,6,2001.0,608.0,0,42,0,0,0,0,0,9
4,70,60.0,9550,1915,1970,0.0,216.0,0.0,540.0,756.0,961,756,0,1717,7,1998.0,642.0,0,35,272,0,0,0,0,2
5,60,84.0,14260,2000,2000,350.0,655.0,0.0,490.0,1145.0,1145,1053,0,2198,9,2000.0,836.0,192,84,0,0,0,0,0,12


In [833]:
# lets try to check the percentage of missing values,unique values,percentage of one catagory values and type against each column.
def statistics(df):
    stats = []
    for col in df.columns:
        stats.append((col, df[col].nunique(), df[col].isnull().sum(), df[col].isnull().sum() * 100 / df.shape[0], df[col].dtype))

    stats_df = pd.DataFrame(stats, columns=['Feature', 'Unique_values', 'Missing values', 'Percentage of Missing Values', 'Data Type'])
    stats_df.set_index('Feature', drop=True, inplace=True)
    stats_df.drop(stats_df[stats_df['Missing values'] == 0].index, axis=0, inplace=True)
    stats_df.sort_values('Percentage of Missing Values', ascending=False, inplace=True)
    return stats_df

statistics(df)

,Unique_values,Missing values,Percentage of Missing Values,Data Type
Feature,,,,
LotFrontage,128,486,16.649538,float64
GarageYrBlt,103,159,5.447071,float64
MasVnrArea,444,23,0.787941,float64
BsmtFinSF1,991,1,0.034258,float64
BsmtFinSF2,272,1,0.034258,float64
BsmtUnfSF,1135,1,0.034258,float64
TotalBsmtSF,1058,1,0.034258,float64
GarageArea,603,1,0.034258,float64


In [834]:
# Filling NAN with mean values
for i in df.columns:
    df[i].fillna(df[i].mean(), inplace=True)

In [835]:
statistics(df)

,Unique_values,Missing values,Percentage of Missing Values,Data Type
Feature,,,,


In [836]:
data = df.copy()

## Outlier removal wit STD Mean and model building

One of the ways we can remove outliers is remove any data points that are beyond 2 standard deviation from mean. Which means we can come up with following upper and lower bounds

In [837]:
# Function to display detected outliers
def outliers_statistics(df):
    stats = []
    for col in df.columns:
        stats.append((col, df[col].isnull().sum(), df[col].isnull().sum() * 100 / df.shape[0]))

    stats_df = pd.DataFrame(stats, columns=['Feature', 'Outliers', 'Percentage of Outliers'])
    stats_df.set_index('Feature', drop=True, inplace=True)
    stats_df.drop(stats_df[stats_df['Outliers'] == 0].index, axis=0, inplace=True)
    stats_df.sort_values('Percentage of Outliers', ascending=False, inplace=True)
    return stats_df

outliers_statistics(df)

,Outliers,Percentage of Outliers
Feature,,


In [838]:
# Function to detect outliers with mean standard deviation
def std_mean(feature, std_thresh=2, df=df):
    upper_limit = df[feature].mean() + std_thresh*df[feature].std()
    lower_limit = df[feature].mean() - std_thresh*df[feature].std()
    index = df[(df[feature] > upper_limit) | (df[feature] < lower_limit)][feature].index
    df[feature].loc[index] = np.nan
    return df

In [839]:
# detecting outliers in feature
for i in data.columns:
    df = std_mean(i, 2, df)

# Displaying outliers
outliers_statistics(df)

,Outliers,Percentage of Outliers
Feature,,
GarageArea,234,8.016444
MSSubClass,207,7.091470
ScreenPorch,199,6.817403
LotFrontage,198,6.783145
EnclosedPorch,193,6.611853
BsmtUnfSF,163,5.584104
BsmtFinSF2,161,5.515588
TotRmsAbvGrd,157,5.378554
TotalBsmtSF,156,5.344296


In [840]:
# Total Number of Outliers in all features
outliers_statistics(df).Outliers.sum()

3050

In [841]:
# Dropping outliers
df.dropna(inplace=True)

# Checking outliers
outliers_statistics(df)

,Outliers,Percentage of Outliers
Feature,,


In [842]:
def prepare_data(df, null_in_sale=50):
    # Taking out categorical features for remaning data
    new_df = main_data.copy()
    new_df = new_df.loc[list(df.index), :]
    new_df[num_cols] = df.values

    # Dropping categorical features in which more than 50 percent null values are present
    stat = statistics(new_df)
    new_df.drop(stat[stat["Percentage of Missing Values"] > null_in_sale].index, axis=1, inplace=True)

    # Filling null values with mode in categorical features
    for i in statistics(new_df).index[1:]:
        new_df[i].fillna(new_df[i].mode()[0], inplace=True)

    # Encoding categorical feature
    from sklearn.preprocessing import LabelEncoder
    encoder = LabelEncoder()

    for i in new_df.select_dtypes(include=['O']).columns:
        new_df[i] = encoder.fit_transform(new_df[i])

    # Splitting data into train and test part
    train = new_df[new_df.SalePrice.notnull()]
    test = new_df[new_df.SalePrice.isnull()]

    # Separating data into input and output data
    X_train, y_train = train.drop("SalePrice", axis=1), train.SalePrice
    X_test, y_test = test.drop("SalePrice", axis=1), test.SalePrice
    
    return X_train, X_test, y_train, y_test

In [843]:
# Preparing data for model
X_train, X_test, y_train, y_test = prepare_data(df)

In [844]:
# Arrays to store details
method_name = []
model_score = []

In [845]:
# Building model
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor()
model.fit(X_train, y_train)
score = model.score(X_train, y_train)
method_name.append("STD Mean")
model_score.append(score)
print(f"Model Accuracy : {score}")

Model Accuracy : 0.9012487774891377


## Outlier removal wit STD Median and model building

In [846]:
df = data.copy()

In [847]:
# Function to detect outliers with mean standard deviation
def std_mean(feature, std_thresh=2, df=df):
    upper_limit = df[feature].median() + std_thresh*df[feature].std()
    lower_limit = df[feature].median() - std_thresh*df[feature].std()
    index = df[(df[feature] > upper_limit) | (df[feature] < lower_limit)][feature].index
    df[feature].loc[index] = np.nan
    return df

In [848]:
# detecting outliers in feature
for i in data.columns:
    df = std_mean(i, 2, df)

# Displaying outliers
outliers_statistics(df)

,Outliers,Percentage of Outliers
Feature,,
2ndFlrSF,440,15.073655
YearRemodAdd,375,12.846865
WoodDeckSF,302,10.346009
MasVnrArea,250,8.564577
EnclosedPorch,239,8.187736
GarageArea,228,7.810894
ScreenPorch,224,7.673861
BsmtUnfSF,210,7.194245
MSSubClass,207,7.091470


In [849]:
# Total Number of Outliers in all features
outliers_statistics(df).Outliers.sum()

4296

In [850]:
# Dropping outliers
df.dropna(inplace=True)

# Checking outliers
outliers_statistics(df)

,Outliers,Percentage of Outliers
Feature,,


In [851]:
# Preparing data for model
X_train, X_test, y_train, y_test = prepare_data(df, 52)

In [852]:
# Building model
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor()
model.fit(X_train, y_train)
score = model.score(X_train, y_train)
method_name.append("STD Median")
model_score.append(score)
print(f"Model Accuracy : {score}")

Model Accuracy : 0.9758636465376592


With **Mean** method we detected **2605** outliers in whole dataset with standard deviation 2. And model's accuracy is 98 % percent on such data.

With **Median** method we detected **3495** outliers in whole dataset with standard deviation 2. Model accuracy is 1 percent less than the mean method. Here the accuracy is around 97 percent.

## Outlier removal with EllipticEnvelope and model building

In [853]:
def set_data(df):
    # Filling NAN with mean values
    for i in num_cols:
        df[i].fillna(df[i].mean(), inplace=True)
    
    # Filling null values with mode in categorical features
    for i in statistics(df).index[1:]:
        df[i].fillna(df[i].mode()[0], inplace=True)
    
    # Encoding categorical feature
    from sklearn.preprocessing import LabelEncoder
    encoder = LabelEncoder()

    for i in df.select_dtypes(include=['O']).columns:
        df[i] = encoder.fit_transform(df[i])
    return df

In [854]:
def split_data(df):
    # Splitting data into train and test part
    train = df[df.SalePrice.notnull()]
    test = df[df.SalePrice.isnull()]

    # Separating data into input and output data
    X_train, y_train = train.drop(["SalePrice", "outlier"], axis=1), train.SalePrice
    X_test, y_test = test.drop(["SalePrice", "outlier"], axis=1), test.SalePrice
    
    return X_train, X_test, y_train, y_test

In [855]:
df = set_data(main_data)

In [856]:
from sklearn.covariance import EllipticEnvelope
ellip = EllipticEnvelope()
ellip.fit(df.drop("SalePrice", axis=1))

In [ ]:
# predict returns 1 for an inlier and -1 for an outlier
y_pred = ellip.predict(df.drop("SalePrice", axis=1))
df["outlier"] = y_pred

# Dropping outliers
df = df[df.outlier == 1]

In [ ]:
X_train, X_test, y_train, y_test = split_data(df)

In [ ]:
# Building model
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor()
model.fit(X_train, y_train)
score = model.score(X_train, y_train)
method_name.append("EllipticEnvelope")
model_score.append(score)
print(f"Model Accuracy : {score}")

Model Accuracy : 0.9156103113941547


## Outlier removal with LocalOutlierFactor and model building

In [ ]:
df = set_data(main_data)

In [ ]:
from sklearn.neighbors import LocalOutlierFactor
fac = LocalOutlierFactor()
fac.fit(df.drop("SalePrice", axis=1))

LocalOutlierFactor()

In [ ]:
# predict returns 1 for an inlier and -1 for an outlier
y_pred = fac.fit_predict(df.drop("SalePrice", axis=1))
df["outlier"] = y_pred

# Dropping outliers
df = df[df.outlier == 1]

In [ ]:
X_train, X_test, y_train, y_test = split_data(df)

In [ ]:
# Building model
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor()
model.fit(X_train, y_train)
score = model.score(X_train, y_train)
method_name.append("LocalOutlierFactor")
model_score.append(score)
print(f"Model Accuracy : {score}")

KeyboardInterrupt: 

## Outlier removal with IsolationForest and model building

In [ ]:
df = set_data(main_data)

In [ ]:
from sklearn.ensemble import IsolationForest
iso = IsolationForest()
iso.fit(df.drop("SalePrice", axis=1))

IsolationForest()

In [ ]:
# predict returns 1 for an inlier and -1 for an outlier
y_pred = iso.fit_predict(df.drop("SalePrice", axis=1))
df["outlier"] = y_pred

# Dropping outliers
df = df[df.outlier == 1]

In [ ]:
X_train, X_test, y_train, y_test = split_data(df)

In [ ]:
# Building model
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor()
model.fit(X_train, y_train)
score = model.score(X_train, y_train)
method_name.append("IsolationFactor")
model_score.append(score)
print(f"Model Accuracy : {score}")

Model Accuracy : 0.9105319355570242


# Comparison

In [ ]:
comp = pd.DataFrame({"Method": method_name, "Model's Performances": model_score})
comp.sort_values("Model's Performances", ascending=False, inplace=True)
comp

,Method,Model's Performances
1,STD Median,0.974643
2,EllipticEnvelope,0.915983
3,LocalOutlierFactor,0.914069
4,IsolationFactor,0.910532
0,STD Mean,0.899990


From all of this we can see that our method **STD Mean** is performing best and than **STD Median**. Our proposed methods are performing much better than built-in methods.